In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import json

# Load data from JSON file
with open('data.json') as f:
    data = json.load(f)

# Extract specs and edges from JSON data
specs = data['specs']
edges = data['edges']

# Create a directed graph
G = nx.DiGraph()

# Add specs with attributes
for node, attr in specs.items():
    G.add_node(node, **attr)

# Add edges with attributes
for edge, description in edges.items():
    start, end = edge.split('-')
    G.add_edge(start, end, description=description)

# Function to find direct and indirect specs and print relationships
def find_connections(graph, start_node):
    direct_successors = list(graph.successors(start_node))
    direct_predecessors = list(graph.predecessors(start_node))
    all_connected = nx.descendants(graph, start_node)  # This gets all specs that can be reached from start_node

    indirect_specs = all_connected - set(direct_successors)

    print(f"Analysed spec: '{start_node}'")
    
    print(f"\nDirect successors of '{start_node}': {direct_successors}")
    for succ in direct_successors:
        print(f"  {start_node} -> {succ}: {graph[start_node][succ]['description']}")

    print(f"\nDirect predecessors of '{start_node}': {direct_predecessors}")
    for pred in direct_predecessors:
        print(f"  {pred} -> {start_node}: {graph[pred][start_node]['description']}")

    print(f"\nIndirectly connected specs from '{start_node}': {indirect_specs}")
    for ind_spec in indirect_specs:
        paths = list(nx.all_simple_paths(graph, start_node, ind_spec))
        for path in paths:
            print(f"  Path: {' -> '.join(path)}\n")
    return direct_successors, direct_predecessors, indirect_specs

# Example usage
direct_successors, direct_predecessors, indirect_specs = find_connections(G, "Orbit")


Analysed spec: 'Orbit'

Direct successors of 'Orbit': ['FieldOfView', 'Lifetime', 'Radiation']
  Orbit -> FieldOfView: The orbit determines the field of view for instruments, influencing what areas of space or planetary surfaces can be observed.
  Orbit -> Lifetime: The orbit affects the spacecraft's operational lifetime because factors like radiation exposure and atmospheric drag vary with altitude and orbital path.
  Orbit -> Radiation: The chosen orbit affects radiation exposure levels, with higher orbits typically encountering more intense radiation fields.

Direct predecessors of 'Orbit': ['Communication', 'Propulsion']
  Communication -> Orbit: Communication needs affect the choice of orbit, as certain orbits provide better line-of-sight and signal strength for data transmission.
  Propulsion -> Orbit: Propulsion system capabilities affect the chosen orbit, as more powerful propulsion allows for higher or more stable orbits.

Indirectly connected specs from 'Orbit': {'Shielding'}